In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
import tensorflow as tf

In [2]:
# Baca dataset
df = pd.read_csv('cuaca.csv')

# Label encoding untuk kolom 'Weather'
label_encoder = LabelEncoder()
df['Weather'] = label_encoder.fit_transform(df['Weather'])

# Pilih fitur dan target
X = df[['Hum', 'Temp']]
y = df['Weather']

In [3]:
# Bagi dataset menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model regresi linear
model = LinearRegression()

# Latih model
model.fit(X_train, y_train)

LinearRegression()

In [4]:
# Load model
loaded_model = load_model('linear_model.h5')

# Evaluasi model pada set pengujian
y_test_pred = loaded_model.predict(X_test)

# Evaluasi model
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)
print('Testing Set:')
print(f'Mean Squared Error: {mse_test}')
print(f'R-squared: {r2_test}')

OSError: No file or directory found at linear_model.h5

In [ ]:
# Konversi model Keras ke format TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Simpan model TFLite
with open('linear_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
# Uji model TFLite dengan nilai acak
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Masukkan nilai acak sebagai input
input_data = np.random.randn(1, 2).astype(np.float32)
interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_data)

# Jalankan inferensi
interpreter.invoke()

# Dapatkan hasil prediksi
output_data = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

# Tampilkan hasil prediksi untuk nilai acak
print('\nRandom Data (TFLite):')
print(pd.DataFrame(input_data, columns=['Hum', 'Temp']))
print('\nPredicted Weather (TFLite):')
print(label_encoder.inverse_transform(np.round(output_data).astype(int)))

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load dataset
dataset = pd.read_csv('cuaca.csv')

# Select features (X) and target (Y)
X = dataset[['Hum', 'Temp']]
Y = dataset['Weather']

# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalize data using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train linear regression model
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, Y_train)

# Create and train a simple Keras model
keras_model = Sequential()
keras_model.add(Dense(1, input_dim=2, kernel_initializer='zeros', bias_initializer='zeros'))

# Set the weights for the Keras model
keras_model.layers[0].set_weights([linear_model.coef_.reshape(2, 1), linear_model.intercept_])

# Save the Keras model
keras_model.save('linear_regression_model.h5')

# Make predictions using Keras model
keras_predictions = keras_model.predict(X_test_scaled).flatten()

# Make predictions using linear regression model
linear_predictions = linear_model.predict(X_test_scaled)

# Evaluate mean squared error
mse_keras = mean_squared_error(Y_test, keras_predictions)
mse_linear = mean_squared_error(Y_test, linear_predictions)

print("Mean Squared Error Keras:", mse_keras)
print("Mean Squared Error Linear Regression:", mse_linear)


ValueError: Layer dense_1 weight shape (1,) is not compatible with provided weight shape ().